In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error as mse
import time
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans
import sys
import os
import time
import pickle
from numba import jit
from scipy.optimize import rosen
import matplotlib.pyplot as plt

def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def get_errors(x_, y_):
    return [np.sqrt(mse(x_, y_)), np.mean(np.abs(x_ - y_)), np.max(np.abs(x_ - y_))]

@jit
def simple_cov(_x, _y):
    return np.mean((_x-np.mean(_x))*(_y-np.mean(_y)), axis = 1)

def get_mcdues(X):
    stds = np.zeros((X.shape[0], params['T']), dtype = float)
    for cnt_ in range(params['T']):
        stds[:, cnt_] = np.ravel(sess.run(y, feed_dict={x: X, 
                                                        keep_probability_: .5}))
    return np.std(stds, axis = 1)

def get_stds(X):
    stds = np.zeros((X.shape[0], T), dtype = float)
    for cnt_ in range(T):
        stds[:, cnt_] = np.ravel(sess.run(y, feed_dict={x: X, 
                                                        keep_probability_: .5}))
    return stds

@jit
def compute_block_inv(A_inv, B, C, D):
    H = D - C.dot(A_inv).dot(B)
    H_inv = 1./ H
    a00 = A_inv + H_inv * A_inv.dot(B).dot(C).dot(A_inv)
    a01 = -A_inv.dot(B) * H_inv
    a10 = -H_inv * C.dot(A_inv)
    a11 = H_inv
    
    return np.block([[a00, a01.reshape(-1, 1)],
                    [a10.reshape((1, -1)), a11[0]]])

params = pd.read_pickle("params.pickle")

df = pd.read_csv('slice_localization_data.csv')
df.drop(['patientId'], axis = 1, inplace = True)
target_label = 'reference'

df = df.sample(frac=1).reset_index(drop=True) # TODO: save these indices
targets = df[target_label].values    
df.drop([target_label], axis = 1, inplace = True)

N_total = len(df)
df = df.iloc[params['df_index']]
N_train, N_test, N_val, N_pool = params['N_train'], params['N_test'], params['N_val'], params['N_pool']


X_train = df[:N_train].values
y_train = targets[:N_train][:, None]
X_test = df[N_train:(N_train + N_test)].values
y_test = targets[N_train:(N_train + N_test)][:, None]
X_pool = df[-N_pool:].values
y_pool = targets[-N_pool:][:, None]
X_val = df[-(N_val+N_pool):-N_pool].values
y_val = targets[-(N_val+N_pool):-N_pool][:, None]

print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_train.shape)
print('shapes:', X_pool.shape, y_pool.shape)
print('shapes:', X_val.shape, y_val.shape)


ndim = params['ndim']
# layers = [64,32]
layers = params['layers']

learning_rate_decay = params['learning_rate_decay']
start_learning_rate = params['start_learning_rate']
learning_rate_schedule_epochs = params['learning_rate_schedule_epochs']

tf.reset_default_graph()

# placeholders
x = tf.placeholder(tf.float32, [None, ndim])
y_ = tf.placeholder(tf.float32, [None, 1])

learning_rate_ = tf.placeholder(tf.float32)
forces_coeff_ = tf.placeholder(tf.float32)
keep_probability_ = tf.placeholder(tf.float32, name='keep_probability')
l2_reg_ = tf.placeholder(tf.float32, name='l2reg')

# weights
W1 = tf.Variable(tf.truncated_normal([ndim, layers[0]], stddev=(2/ndim)**.5))
b1 = tf.Variable(tf.truncated_normal([layers[0]],  stddev=.1))
h1 = tf.nn.relu(tf.matmul(x, W1) + b1)
h_drop1 = tf.nn.dropout(h1, keep_probability_, noise_shape = [1,layers[0]])

Ws = [W1]; bs = [b1]; hs = [h_drop1]
for cnt_layer in range(1, len(layers)):
    Ws.append(tf.Variable(tf.truncated_normal([layers[cnt_layer - 1], layers[cnt_layer]], 
                                              stddev=(2/layers[cnt_layer - 1])**.5)))
    bs.append(tf.Variable(tf.truncated_normal([layers[cnt_layer]],  stddev=.1)))
    hs.append(tf.nn.dropout(tf.nn.relu(tf.matmul(hs[-1], Ws[-1]) + bs[-1]), keep_probability_,
                            noise_shape = [1,layers[cnt_layer]]))

Ws.append(tf.Variable(tf.truncated_normal([layers[-1], 1], stddev=.1)))
bs.append(tf.Variable(tf.truncated_normal([1],  stddev=.1)))

# funcs
y = tf.matmul(hs[-1], Ws[-1]) + bs[-1]

l2_regularizer = sum(tf.nn.l2_loss(Wxxx) for Wxxx in Ws) 

mse_e = tf.losses.mean_squared_error(predictions = y, labels = y_)
loss = mse_e + l2_reg_*l2_regularizer

#train_step = tf.train.AdamOptimizer(learning_rate=learning_rate_).minimize(loss)

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = start_learning_rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           learning_rate_schedule_epochs, learning_rate_decay, staircase=True)

lr_fun = lambda: learning_rate
min_lr = lambda: tf.constant(1e-5)
actual_lr = tf.case([(tf.less(learning_rate, tf.constant(1e-5)), min_lr)], default=lr_fun)

train_step = tf.train.AdamOptimizer(learning_rate=actual_lr).minimize(loss, global_step=global_step)

try:
    sess.close()
except:
    pass

init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
epoch = 0
data = []

batch_size = params['batch_size']
init_epochs = params['init_epochs']
keep_prob = params['keep_prob']
l2_reg = params['l2_reg']

al_steps = params['al_steps']
uptrain_epochs = params['uptrain_epochs']
mandatory_uptrain_epochs = params['mandatory_uptrain_epochs']
sample_each_step = params['sample_each_step']
T = params['T']

early_stopping_window = params['early_stopping_window']
max_warnings = params['max_warnings']
early_stopping_check_step = params['early_stopping_check_step']

gpnn_max_train = params['gpnn_max_train']
diag_eps = params['diag_eps']

data_columns = params['data_columns']

X_train_current = X_train.copy()
y_train_current = y_train.copy()
X_pool_current = X_pool.copy()
y_pool_current = y_pool.copy()

fname_identifier = params['fname_identifier']
w8s_folder = "/Users/r.ushakov/Desktop/NNGP/integrated_gain_max_variance/init_w8s/"
saver.restore(sess, w8s_folder + "init_" + params['fname_identifier'] + ".ckpt")
print("Init model restored")

@jit
def dodot(Q, W):
    return np.dot(np.dot(Q.T, W), Q)[0][0]

@jit
def fxjit(x_cnt_):
    pool_sample = stds[(gpnn_max_train + x_cnt_), :]
    #t = time.time()
    Q = simple_cov(stds[:gpnn_max_train, :], pool_sample)[:, None]
    Q = Q.ravel()
    #print('Q in', np.round(time.time()-t, 2)); t=time.time()
    new_K_cov[-1, :-1] = Q
    new_K_cov[:-1, -1] = Q
    new_K_cov[-1, -1] = np.var(pool_sample)
    #print('Q insert in', np.round(time.time()-t, 2)); t=time.time()
    new_K_cov_inv = compute_block_inv(K_train_cov_inv,
                                      Q.reshape((-1, 1)),
                                      Q.reshape((1, -1)), 
                                      np.var(pool_sample) + 0.001)
    #print('new_K_cov_inv in', np.round(time.time()-t, 2)); t=time.time()
    indices = list(range(gpnn_max_train)) + [gpnn_max_train + x_cnt_]
    si = stds[indices]
    ### count sigma(v | X + x_from_pool) with extended 
    ### cov matrix for each v in vs
    extended_sigmas = np.zeros((len(y_vs),))
    for cnt_ in range(len(y_vs)):
        vs_sample = y_vs[cnt_, :]
        Q = simple_cov(si, vs_sample)[:, None]
        KK = np.var(vs_sample)
        #sigma = KK + params['diag_eps'] - np.dot(np.dot(Q.T, new_K_cov_inv), Q)[0][0]
        sigma = KK - dodot(Q, new_K_cov_inv) + diag_eps
        extended_sigmas[cnt_] = np.sqrt(sigma)
    return extended_sigmas
#     #print('integrate in', np.round(time.time()-t, 2)); t=time.time()
#     current_diff = np.array(sigmas) - np.array(extended_sigmas)
#     #print('misc in', np.round(time.time()-t, 2)); t=time.time()
#     return current_diff.sum()

extended_sigmas_list = []
sigmas_list = []
np.random.seed(0)

points_int_list = [10, 50, 100, 200, 300]

for points_to_integrate in points_int_list:

    t = time.time()
    perm = np.random.permutation(range(len(X_train_current)))
    random_train_inds = perm[:gpnn_max_train]
    random_train_samples = X_train_current[random_train_inds]
    train_and_pool_samples = np.concatenate([random_train_samples, X_pool_current])    
    stds = get_stds(train_and_pool_samples)
    K_train_cov = np.cov(stds[:gpnn_max_train, :], ddof = 0)
    K_train_cov_inv = np.linalg.inv(K_train_cov + diag_eps * np.eye(gpnn_max_train))

    vs = X_pool_current[-points_to_integrate:,:]
    y_vs = get_stds(vs)
#     print('Work on sigmas at #', al_iters)
    ### sigma(v | X) for each v in vs
    sigmas = np.zeros((len(y_vs),))
    for cnt_ in range(len(vs)):
        vs_sample = y_vs[cnt_, :]
        Q = simple_cov(stds[:gpnn_max_train], vs_sample)[:, None]
        KK = np.var(vs_sample)
        sigma = KK - np.dot(np.dot(Q.T, K_train_cov_inv), Q)[0][0]
        sigmas[cnt_] = np.sqrt(sigma)

    diffs_integral = np.zeros(X_pool_current.shape[0])
    new_K_cov = np.zeros((gpnn_max_train + 1, gpnn_max_train + 1))
    new_K_cov[:gpnn_max_train, :gpnn_max_train] = K_train_cov

    for x_cnt_ in range(len(X_pool_current)-points_to_integrate):
        extended_sigmas = fxjit(x_cnt_)
        if x_cnt_ % 10 == 0:
            print(x_cnt_, end = '|')
            
    sigmas_list.append(sigmas)
    extended_sigmas_list.append(extended_sigmas)
    
for i in range(7):
    idx_point = i
    ext_point_idx_values = [arr[idx_point] for arr in extended_sigmas_list]

    plt.figure(figsize=(8, 6))
    plt.plot(points_int_list, ext_point_idx_values, label="UE for point {}".format(idx_point))
    plt.xlabel("points to integrate")
    plt.ylabel("ue")
    plt.legend()
    plt.grid()
    plt.savefig("ue_point_{}.png".format(i))